### Instruction
Before using this file, please make sure that these tasks are completed:
- Convert all the images to the target size using `transform_image.py`
- Annotate the test set and the train set using `labelImg` module into `pascal` form
- Convert the resulting `.xml` file into `.csv` using `xml_to_csv.py`




#### Mounting Project data from Gdrive

In [1]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/gdrive')

Drive not mounted, so nothing to flush and unmount.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


#### Setting Tensorflow Object Detection module

In [2]:
%%time

!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

%cd /content/gdrive/My Drive/SUTD/cds_nude_censoring/models/research

!protoc object_detection/protos/*.proto --python_out=.
%set_env PYTHONPATH=/content/gdrive/My Drive/SUTD/cds_nude_censoring/models/research:/content/gdrive/My Drive/SUTD/cds_nude_censoring/models/research/slim
!python object_detection/builders/model_builder_test.py

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.16-2~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 32 not upgraded.
Need to get 1,818 kB of archives.
After this operation, 7,688 kB of additional disk space will be used.
Get:1 htt

#### Generating Tensorflow Record

In [3]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

os.chdir('../../')
print("Current working directory: ", os.getcwd()) # Should be /content/gdrive/My Drive/SUTD/cds_nude_censoring

Current working directory:  /content/gdrive/My Drive/SUTD/cds_nude_censoring


In [0]:
def class_text_to_int(row_label):
    if row_label == 'breast':
        return 1
    elif row_label == 'breasts':
        return 2
    elif row_label == 'penis':
        return 3
    elif row_label == 'vagina':
        return 4
    elif row_label == 'fucking':
        return 5
    else:
        None

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def generate_tfrecord(csv_input, output_path, image_dir):
    writer = tf.python_io.TFRecordWriter(output_path)
    path = os.path.join(image_dir)
    examples = pd.read_csv(csv_input)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))

In [5]:
# %%time
# generate_tfrecord('data/test_set_labels.csv', 'test.record', 'images/test_set')

Successfully created the TFRecords: /content/gdrive/My Drive/SUTD/cds_nude_censoring/test.record
CPU times: user 181 ms, sys: 22.6 ms, total: 204 ms
Wall time: 24.8 s


In [6]:
# %%time
# generate_tfrecord('data/train_set_labels.csv', 'train.record', 'images/train_set')

Successfully created the TFRecords: /content/gdrive/My Drive/SUTD/cds_nude_censoring/train.record
CPU times: user 309 ms, sys: 29.8 ms, total: 339 ms
Wall time: 46.2 s


#### Training the Model
Path to model is stored in `pipeline2.config`. Follow instruction in the file to know which paths to change.

Trained model will be stored in the mentioned model directory in as a checkpoint file. `model.ckpt-XXXX.data-YYYYY-of-ZZZZZ`

The highest number of `XXXX` refers to the latest model. Each models takes approximately 5 minutes to train.

In [4]:
os.chdir('models/research/object_detection')
print('Current working directory: ', os.getcwd()) # Should be /content/gdrive/My Drive/SUTD/cds_nude_censoring/models/research/object_detection

Current working directory:  /content/gdrive/My Drive/SUTD/cds_nude_censoring/models/research/object_detection


In [8]:
# %%time
# !python model_main.py --logtostderr --model_dir=training2/ --pipeline_config_path=training2/pipeline2.config

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W1204 16:30:17.048488 139842327984000 module_wrapper.py:139] From /content/gdrive/My Drive/SUTD/cds_nude_censoring/models/research/object_detection/utils/config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1204 16:30:17.252488 139842327984000 module_wrapper.py:139] From /content/gdrive/My Drive/SUTD/cds_nude_censoring/models/research/object_detection/model_lib.py:628: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.

W1204 16:30:17.252739 139842327984000 model_lib.py:629] Forced number of epochs for all eval validations to be 1.

W1204

#### Generate Inference Graph
Replace the path to `model.ckpt-XXXX` with the highest checkpoint number. Ignore the `.data-00000-of-00001` from the input.)

This code will create a new folder called `inference_graph` containing `frozen_inference_graph` based on the selected model that will be used to predict.


In [5]:
%%time
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training2/pipeline2.config --trained_checkpoint_prefix training2/model.ckpt-3144 --output_directory inference_graph

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W1204 17:41:14.016506 140055175571328 module_wrapper.py:139] From export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1204 17:41:14.444301 140055175571328 module_wrapper.py:139] From /content/gdrive/My Drive/SUTD/cds_nude_censoring/models/research/object_detection/exporter.py:402: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W1204 17:41:14.444845 140055175571328 module_wrapper.py:139] From /content/gdrive/My Drive/SUTD/cds_nude_censoring/models/research/object_detection/exporter.py:121: The name tf.placeholder is deprecated. Pl

#### Object Detection
Current working directory should be located in `/content/gdrive/My Drive/SUTD/cds_nude_censoring/models/research/object_detection`

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import glob
import time

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

In [0]:
# This is needed to display the images.
%matplotlib inline

In [0]:
from utils import label_map_util
from utils import visualization_utils as vis_util

In [0]:
MODEL_DIR = 'inference_graph'
PATH_TO_FROZEN_GRAPH = MODEL_DIR + '/frozen_inference_graph.pb'
PATH_TO_LABELS = 'training2/labelmap.pbtxt'

In [0]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [0]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [0]:
def run_inference_for_single_image(image, graph):
    if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
    image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

    # Run inference
    output_dict = sess.run(tensor_dict,
                            feed_dict={image_tensor: np.expand_dims(image, 0)})

    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
        'detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [0]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [14]:
os.getcwd()

'/content/gdrive/My Drive/SUTD/cds_nude_censoring/models/research/object_detection'

In [0]:
CONTENT_PATH = '../../../../../../../*.jpg'
CDS_NUDE_CENSORING_IMAGE_PATH = '../../../images/test_set/*.jpg'

with detection_graph.as_default():
    with tf.Session() as sess:
        ops = tf.get_default_graph().get_operations()
        all_tensor_names = {output.name for op in ops for output in op.outputs}
        tensor_dict = {}
        for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
          ]:
            tensor_name = key + ':0'
            if tensor_name in all_tensor_names:
                tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
        counter = 1
        for image_path in glob.glob(CDS_NUDE_CENSORING_IMAGE_PATH):
            t1 = time.time()

            print(image_path)
            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            output_dict = run_inference_for_single_image(image_np, detection_graph)
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                output_dict['detection_boxes'],
                output_dict['detection_classes'],
                output_dict['detection_scores'],
                category_index,
                instance_masks=output_dict.get('detection_masks'),
                use_normalized_coordinates=True,
                line_thickness=4)
            plt.figure(figsize=(400, 300))
            plt.imshow(image_np)
            plt.savefig('../../../../../test_' + str(counter) + '.png')
            print('Image saved as ./test_' + str(counter) + '.png, time taken: ', time.time() - t1, 's')
            counter += 1
        print('Completed...')
        sess.close()

../../../images/test_set/0-3.jpg
Saving image as ./test_1.png
../../../images/test_set/01-10.jpg
Saving image as ./test_2.png
../../../images/test_set/004.jpg
Saving image as ./test_3.png
../../../images/test_set/001_1000.jpg
Saving image as ./test_4.png
../../../images/test_set/003.jpg
Saving image as ./test_5.png
../../../images/test_set/003x.jpg
Saving image as ./test_6.png
../../../images/test_set/01-11.jpg
Saving image as ./test_7.png
../../../images/test_set/00003-9ddb1ef6.jpg
Saving image as ./test_8.png
../../../images/test_set/01-1.jpg


In [20]:
#@title Default title text
import cv2
cap = cv2.VideoCapture(0)
# cap = cv2.imread('../../../images/test_set/0-3.jpg')
try:
    with detection_graph.as_default():
        with tf.Session() as sess:
                # Get handles to input and output tensors
                ops = tf.get_default_graph().get_operations()
                all_tensor_names = {output.name for op in ops for output in op.outputs}
                tensor_dict = {}
                for key in [
                  'num_detections', 'detection_boxes', 'detection_scores',
                  'detection_classes', 'detection_masks'
                ]:
                    tensor_name = key + ':0'
                    if tensor_name in all_tensor_names:
                        tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                      tensor_name)

                while True:
                    ret, image_np = cap.read()
                    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                    image_np_expanded = np.expand_dims(image_np, axis=0)
                    # Actual detection.
                    output_dict = run_inference_for_single_image(image_np, detection_graph)
                    # Visualization of the results of a detection.
                    vis_util.visualize_boxes_and_labels_on_image_array(
                        image_np,
                        output_dict['detection_boxes'],
                        output_dict['detection_classes'],
                        output_dict['detection_scores'],
                        category_index,
                        instance_masks=output_dict.get('detection_masks'),
                        use_normalized_coordinates=True,
                        line_thickness=8)
                    cv2.imshow('object_detection', cv2.resize(image_np, (800, 600)))
                    if cv2.waitKey(25) & 0xFF == ord('q'):
                        cap.release()
                        cv2.destroyAllWindows()
                        break
except Exception as e:
    print(e)
    cap.release()

int() argument must be a string, a bytes-like object or a number, not 'NoneType'
